In [1]:
import os

In [2]:
%pwd

'd:\\Nikku\\End to End Project\\Healthy.ai\\research\\Diet_Recommendation'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\Nikku\\End to End Project\\Healthy.ai'

In [7]:
import pandas as pd

In [13]:
df = pd.read_csv('artifacts\diet_recomendation\data_ingestion\diet_recommendation.csv')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\akash\AppData\Local\Temp\ipykernel_13956\373687862.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('artifacts\diet_recomendation\data_ingestion\diet_recommendation.csv')


In [14]:
df.head()

,Breakfast,Lunch,Snacks,Dinner,Calories,Carbs,Total Fat,Saturated Fat,Protein,Fiber,...,Magnesium,Phosphorus,Vitamin C,Vitamin A,Calcium,Iron,Zinc,Vitamin E,Vitamin K,food_items
0,1,0,1,0,94.64,25.48,0.364,0.182,0.546,4.368,...,9.10,20.02,8.372,10.92,0.3276,4.004,10.92,0.1820,0.0910,Apple
1,1,0,1,0,26.40,6.05,0.220,0.000,0.770,1.100,...,5.50,12.65,5.500,1059.30,0.4895,1.705,7.15,0.2145,0.1100,Apricot
2,1,0,1,0,48.00,2.70,0.600,0.090,0.600,0.600,...,8.70,15.60,3.000,2.10,0.6210,6.300,3.60,0.1800,0.1800,Avocado
3,1,0,1,0,105.02,27.14,0.354,0.118,1.298,3.068,...,31.86,25.96,10.266,1.18,0.1180,0.590,5.90,0.3540,0.1770,Banana
4,1,0,1,0,71.50,17.16,0.715,0.000,1.430,4.290,...,41.47,42.90,41.899,1258.40,2.4310,1.287,77.22,2.2880,1.0868,Black Chokeberry


In [15]:
df.isnull().sum()

Breakfast        0
Lunch            0
Snacks           0
Dinner           0
Calories         0
Carbs            0
Total Fat        0
Saturated Fat    0
Protein          0
Fiber            0
Cholesterol      0
Sodium           0
Sugar            0
Potassium        0
Magnesium        0
Phosphorus       0
Vitamin C        0
Vitamin A        0
Calcium          1
Iron             0
Zinc             0
Vitamin E        0
Vitamin K        0
food_items       0
dtype: int64

In [16]:
df.duplicated().sum()

0

In [17]:
df.columns

Index(['Breakfast', 'Lunch', 'Snacks', 'Dinner', 'Calories', 'Carbs',
       'Total Fat', 'Saturated Fat', 'Protein', 'Fiber', 'Cholesterol',
       'Sodium', 'Sugar', 'Potassium', 'Magnesium', 'Phosphorus', 'Vitamin C',
       'Vitamin A', 'Calcium', 'Iron', 'Zinc', 'Vitamin E', 'Vitamin K',
       'food_items'],
      dtype='object')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Breakfast      717 non-null    int64  
 1   Lunch          717 non-null    int64  
 2   Snacks         717 non-null    int64  
 3   Dinner         717 non-null    int64  
 4   Calories       717 non-null    float64
 5   Carbs          717 non-null    float64
 6   Total Fat      717 non-null    float64
 7   Saturated Fat  717 non-null    float64
 8   Protein        717 non-null    float64
 9   Fiber          717 non-null    float64
 10  Cholesterol    717 non-null    float64
 11  Sodium         717 non-null    float64
 12  Sugar          717 non-null    float64
 13  Potassium      717 non-null    float64
 14  Magnesium      717 non-null    float64
 15  Phosphorus     717 non-null    float64
 16  Vitamin C      717 non-null    float64
 17  Vitamin A      717 non-null    float64
 18  Calcium   

## Entity

In [19]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DietRecomenedDataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict
    

## Configuration Manager

In [20]:
from Healthyai.constants import *
from Healthyai.utils.common import read_yaml,create_directories

In [21]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts])
        
    
    def get_data_validation_config_diet_recommednation(self) ->DietRecomenedDataValidationConfig:
        config = self.config.diet_recommendation_data_validation
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config = DietRecomenedDataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema = schema
        )
        return data_validation_config

##  Components

In [22]:
import os
from Healthyai import logger

In [23]:
class DataValidation:
    def __init__(self, config: DietRecomenedDataValidationConfig):
        self.config = config
     
    # Checking all columns are present in the dataset
    
    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            data = pd.read_csv(self.config.unzip_data_dir)
            all_col = list(data.columns)
            all_schema = self.config.all_schema.keys()
            
            for col in all_col:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"validation status : {validation_status}")
                else:
                    validation_status =True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"validation status : {validation_status}")
            return validation_status
        
        except Exception as e:
            raise e
            
           
    
        

## Pipeline

In [27]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config_diet_recommednation()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-12-05 17:43:47,115: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-05 17:43:47,119: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-05 17:43:47,121: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-05 17:43:47,122: INFO: common: Created directory at : artifacts]
[2023-12-05 17:43:47,123: INFO: common: Created directory at : artifacts/diet_recomendation/data_validation]
